# Multilayer Perceptron - Iris Dataset

Este notebook tem como intenção implementar um multilayer perceptron para classificar o Iris Dataset.

## Importações

Para execução do código é necessária a importação dos pacotes random e pprint, como no bloco executável abaixo.

In [10]:
import random
import math
import copy


Caso deseje obter os mesmos resultados sempre que executar o código, descomente o código abaixo e o execute, para que os valores aleatórios sejam gerados a partir de uma mesma semente

In [11]:
random.seed(28)

## Funções auxiliares

Abaixo estão as funções auxiliares utilizadas para leitura de arquivo, conversão de valores, calcúlos matriciais, etc.

In [12]:
# Load file
def read_data_set(data_set):
    with open(data_set) as data_file:
        data_set = data_file.read().split("\n")
        for i in range(len(data_set)):
            data_set[i] = data_set[i].split(",")

    return data_set


# Change classification to int values
def get_classification_value(classification):
    if(classification == "Iris-setosa"):
        return 0
    if(classification == "Iris-versicolor"):
        return 1
    if(classification == "Iris-virginica"):
        return 2


# Convert string list to float list
def change_string_to_float(string_list):
    float_list = []
    for i in range(len(string_list)):
        float_list.append(float(string_list[i]))
    return float_list


# Matrix multiplication (for Testing)
def matrix_mul_bias(A, B, bias):
    C = []
    for i in range(len(A)):
        C.append([])
        for j in range(len(B[0])):
            C[i].append(0)
    
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                C[i][j] += A[i][k] * B[k][j]
            C[i][j] += bias[j]
    
    return C


# Vector (A) x matrix (B) multiplication
def vec_mat_bias(A, B, bias):
    C = []
    for i in range(len(B[0])):
        C.append(0)
    
    for j in range(len(B[0])):
        for k in range(len(B)):
            C[j] += A[k] * B[k][j]
        C[j] += bias[j]
    
    return C


# Matrix (A) x vector (B) multipilicatoin (for backprop)
def mat_vec(A, B): 
    C = []
    for i in range(len(A)):
        C.append(0)
    
    for i in range(len(A)):
        for j in range(len(B)):
            C[i] += A[i][j] * B[j]
    
    return C


# derivation of sigmoid (for backprop)
def sigmoid(A, deriv=False):
    if deriv: 
        for i in range(len(A)):
            A[i] = A[i] * (1 - A[i])
    else:
        for i in range(len(A)):
            A[i] = 1 / (1 + math.exp(-A[i]))
    return A


## Função principal

A função principal se inicia lendo o arquivo chamado, por padrão, de "iris.txt", contendo o Iris Dataset. Após isso, ele converte suas entradas para valores flutuantes, o divide em dois conjuntos, um de treino e outro de teste e, por fim, separa os resultados dos vetores de teste.

In [13]:
data_set = read_data_set("iris.txt")

# Change string value to numeric
for line in data_set:
    line[4] = get_classification_value(line[4])
    line[:4] = change_string_to_float(line)


# Create a train and a test data)
data_train = data_set
data_test = []

num_max = 15

num_max_0 = 0
num_max_1 = 0
num_max_2 = 0

for t in data_set:
    if(num_max_0 < num_max and t[4] == 0):
        num_max_0 += 1
        data_test.append(t)
    if(num_max_1 < num_max and t[4] == 1):
        num_max_1 += 1
        data_test.append(t)
    if(num_max_2 < num_max and t[4] == 2):
        num_max_2 += 1
        data_test.append(t)


train_set = []
train_result = []
for data in data_train:
    # split the entrance and the result
    train_set.append(data[:4])
    train_result.append(data[4])

test_set = []
test_result = []
for data in data_test:
    # split the entrance and the result
    test_set.append(data[:4])
    test_result.append(data[4])

Feito isso, ele define a taxa de aprendizado, a quantidade de épocas e quantos neurônios cada camada terá. Após isso o programa gera os vetores de pesos e os vetores bias, inicializando seus valores aleatóriamente.

In [14]:
# Define parameter
alpha = 0.01
epoch = 700
neurons = [4, 6, 3] # number of neurons each layer


# Initiate weight and bias with 0 value
weights = []
for i in range(len(neurons) - 1):
    weights.append([])
    for j in range(neurons[i]):
        weights[i].append([])
        for k in range(neurons[i + 1]):
            weights[i][j].append(0)

weight = []
for i in range(len(weights[0])):
    weight.append([])
    for j in range(len(weights[0][i])):
        weight[i].append(weights[0][i][j])

weight_2 = []
for i in range(len(weights[1])):
    weight_2.append([])
    for j in range(len(weights[1][i])):
        weight_2[i].append(weights[1][i][j])


bias_list = []
for i in range(1, len(neurons)):
    bias_list.append([])
    for j in range(neurons[i]):
        bias_list[i-1].append(0)

bias = []
for i in range(len(bias_list[0])):
    bias.append(bias_list[0][i])

bias_2 = []
for i in range(len(bias_list[1])):
    bias_2.append(bias_list[1][i])

# Initiate weight with random between -1.0 ... 1.0
for i in range(neurons[0]):
    for j in range(neurons[1]):
        weight[i][j] = 2 * random.random() - 1

for i in range(neurons[1]):
    for j in range(neurons[2]):
        weight_2[i][j] = 2 * random.random() - 1



Com os valores inicializados, é feito o treinamento. Para cada item do conjunto de treino é feito o Foward Propagation (FP). Com os resultados do FP, calcula-se o erro total da rede. Após isso, é feito o Backward Propagation (BP), atualizando-se os valores dos pesos e dos bias para cada camada, começando pelas últimas. Com isso feito, imprime-se o valor do erro e o processo é repetido até que se tenha completado o número de épocas.

In [15]:
for e in range(epoch):
    cost_total = 0
    for idx, data_list in enumerate(train_set): # Update for each data; SGD


        # Forward propagation
        h_1 = vec_mat_bias(data_list, weight, bias)
        X_1 = sigmoid(h_1)
        h_2 = vec_mat_bias(X_1, weight_2, bias_2)
        X_2 = sigmoid(h_2)



        # Convert to One-hot target
        target = [0] * neurons[-1]
        target[int(train_result[idx])] = 1


        # Cost function, Square Root Eror
        eror = 0
        for i in range(neurons[-1]):
            eror +=  0.5 * (target[i] - X_2[i]) ** 2 
        cost_total += eror

        # Backward propagation
        delta_2 = []
        for j in range(neurons[2]):
            delta_2.append(-1 * (target[j]-X_2[j]) * X_2[j] * (1-X_2[j]))


        for i in range(neurons[1]):
            for j in range(neurons[2]):
                weight_2[i][j] -= alpha * (delta_2[j] * X_1[i])
                bias_2[j] -= alpha * delta_2[j]

        delta_1 = mat_vec(weight_2, delta_2)
        for j in range(neurons[1]):
            delta_1[j] = delta_1[j] * (X_1[j] * (1-X_1[j]))


        # Update weight and bias (layer 1)
        delta_1 = mat_vec(weight_2, delta_2)
        for j in range(neurons[1]):
            delta_1[j] = delta_1[j] * (X_1[j] * (1-X_1[j]))

        for i in range(neurons[0]):
            for j in range(neurons[1]):
                weight[i][j] -=  alpha * (delta_1[j] * data_list[i])
                bias[j] -= alpha * delta_1[j]


    cost_total /= len(train_set)
    if(e % 100 == 0):
        print("Erro da epoca ", e, ": ", cost_total)
    elif(e == epoch - 1):
        print("Erro da ultima epoca: ", cost_total)

Erro da epoca  0 :  0.36269889651659154
Erro da epoca  100 :  0.12903855948861906
Erro da epoca  200 :  0.0632752739116513
Erro da epoca  300 :  0.04148430869867832
Erro da epoca  400 :  0.03227574851071418
Erro da epoca  500 :  0.027355160049427632
Erro da epoca  600 :  0.024326805640054797
Erro da ultima epoca:  0.022310269294145847


Por fim, é realizado o teste para cada um dos elementos no conjunto de testes. Os resultados são impressos na tela na seguinte ordem: os resultados esperados (vindos do Iris Dataset),  a predição (feita pela rede) e a taxa de acerto (acc).

In [16]:
res = matrix_mul_bias(test_set, weight, bias)
res_2 = matrix_mul_bias(res, weight_2, bias_2)

# Get prediction
preds = []
for r in res_2:
    preds.append(max(enumerate(r), key=lambda x:x[1])[0])

for i in range(len(test_result)):
    test_result[i] = int(test_result[i])
# Print prediction
print("Resultado esperado: ", test_result)
print("Predição:", preds)


# Calculate accuration
acc = 0.0
for i in range(len(preds)):
    if preds[i] == int(test_result[i]):
        acc += 1
print(acc / len(preds) * 100, "%")

Resultado esperado:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Predição: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2]
97.77777777777777 %
